In [70]:
import nfl_data_py as nfl
import pandas as pd
import os
import urllib.request
import matplotlib.pyplot as plt
from matplotlib.offsetbox import AnnotationBbox
from matplotlib.offsetbox import OffsetImage

from PIL import Image
import numpy as np
from io import BytesIO
import requests
# ML Libraryies
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
import hvplot.pandas
import plotly.express as px

# Graphing 
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from matplotlib.pyplot import figure

In [71]:
# import data
weekly_nfl = nfl.import_pbp_data([2023])
pd.set_option('display.max_columns', None)
nfl_data = nfl.import_schedules([2023])

#print(weekly_nfl)

2023 done.
Downcasting floats.


In [72]:
pbp_cols = ['game_id', 'fixed_drive_result', 'home_team', 'away_team']
play_by_play = weekly_nfl[pbp_cols]
unique_values = play_by_play['fixed_drive_result'].unique()
print(unique_values)

['Punt' 'Touchdown' 'Field goal' 'Turnover' 'Opp touchdown'
 'Turnover on downs' 'End of half' 'Missed field goal' None 'Safety']


In [73]:
# remove all values that arent scoring plays. 
exclude_values = ['Turnover on downs', 'End of half', 'Missed field goal', 'Turnover', 'Punt']
pbp_df = play_by_play[~play_by_play['fixed_drive_result'].isin(exclude_values)]
pbp_df = pbp_df.drop_duplicates(subset='game_id', keep='first')
pbp_df.head()

,game_id,fixed_drive_result,home_team,away_team
18,2023_01_ARI_WAS,Touchdown,WAS,ARI
190,2023_01_BUF_NYJ,Field goal,NYJ,BUF
369,2023_01_CAR_ATL,Touchdown,ATL,CAR
543,2023_01_CIN_CLE,Field goal,CLE,CIN
684,2023_01_DAL_NYG,Opp touchdown,NYG,DAL


In [74]:
# Use a for loop to iterate through rows
for index, row in pbp_df.iterrows():
    if row['fixed_drive_result'] != 'Field goal':
        pbp_df.at[index, 'fixed_drive_result'] = 'no_fg'
pbp_df.head()

,game_id,fixed_drive_result,home_team,away_team
18,2023_01_ARI_WAS,no_fg,WAS,ARI
190,2023_01_BUF_NYJ,Field goal,NYJ,BUF
369,2023_01_CAR_ATL,no_fg,ATL,CAR
543,2023_01_CIN_CLE,Field goal,CLE,CIN
684,2023_01_DAL_NYG,no_fg,NYG,DAL


In [75]:
# Group the DataFrame by 'home_team'
grouped_data = pbp_df.groupby('home_team')

home_team_df = grouped_data['fixed_drive_result'].value_counts().reset_index(name='count')


home_team_df.head(20)



,home_team,fixed_drive_result,count
0,ARI,no_fg,3
1,ARI,Field goal,2
2,ATL,Field goal,4
3,ATL,no_fg,1
4,BAL,no_fg,6
5,BUF,no_fg,4
6,BUF,Field goal,3
7,CAR,no_fg,3
8,CAR,Field goal,2
9,CHI,no_fg,4


In [76]:
# create df of fgs vs tds for home team
pivoted_data_home = home_team_df.pivot(index='home_team', columns='fixed_drive_result', values='count').fillna(0).reset_index()
# Rename the 'home_team' column to 'team'
pivoted_data_home.rename(columns={'home_team': 'team'}, inplace=True)
# Remove the index and reset it to the default integer index
pivoted_data_home.reset_index(drop=True, inplace=True)

pivoted_data_home.head(12)


fixed_drive_result,team,Field goal,no_fg
0,ARI,2.0,3.0
1,ATL,4.0,1.0
2,BAL,0.0,6.0
3,BUF,3.0,4.0
4,CAR,2.0,3.0
5,CHI,1.0,4.0
6,CIN,1.0,4.0
7,CLE,3.0,3.0
8,DAL,1.0,3.0
9,DEN,3.0,3.0


In [77]:
# make a df for away teams first score


In [78]:
# Group the DataFrame by 'away_team'
grouped_data_away = pbp_df.groupby('away_team')

away_team_df = grouped_data_away['fixed_drive_result'].value_counts().reset_index(name='count')

away_team_df.head()

,away_team,fixed_drive_result,count
0,ARI,no_fg,4
1,ARI,Field goal,2
2,ATL,Field goal,3
3,ATL,no_fg,2
4,BAL,no_fg,4


In [79]:
# create df of fgs vs tds for away team
pivoted_data_away = away_team_df.pivot(index='away_team', columns='fixed_drive_result', values='count').fillna(0).reset_index()
# Rename the 'away_team' column to 'team'
pivoted_data_away.rename(columns={'away_team': 'team'}, inplace=True)
# Remove the index and reset it to the default integer index
pivoted_data_away.reset_index(drop=True, inplace=True)
pivoted_data_away.head(12)

fixed_drive_result,team,Field goal,no_fg
0,ARI,2.0,4.0
1,ATL,3.0,2.0
2,BAL,1.0,4.0
3,BUF,3.0,1.0
4,CAR,1.0,4.0
5,CHI,1.0,5.0
6,CIN,2.0,3.0
7,CLE,0.0,4.0
8,DAL,1.0,5.0
9,DEN,2.0,2.0


In [80]:
first_score_df = pivoted_data_home.merge(pivoted_data_away, on='team', suffixes=('_home', '_away'))
first_score_df['Field_Goal'] = first_score_df['Field goal_home'] + first_score_df['Field goal_away']
first_score_df['No_FG'] = first_score_df['no_fg_home'] + first_score_df['no_fg_away']
first_score_df['Field_Goal%'] = first_score_df['Field_Goal'] / (first_score_df['Field_Goal'] + first_score_df['No_FG']) 
first_score_df['Touchdown_Goal%'] = first_score_df['No_FG'] / (first_score_df['Field_Goal'] + first_score_df['No_FG']) 
first_score_df['Field_Goal_home%'] = first_score_df['Field goal_home'] / (first_score_df['Field goal_home'] + first_score_df['no_fg_home']) 
first_score_df['Field_Goal_away%'] = first_score_df['Field goal_away'] / (first_score_df['Field goal_away'] + first_score_df['no_fg_away'])
columns_to_remove = ['Field goal_home', 'no_fg_home', 'Field goal_away', 'no_fg_away', 'Field_Goal','No_FG']
first_score_df = first_score_df.drop(columns=columns_to_remove)

first_score_df.head()

fixed_drive_result,team,Field_Goal%,Touchdown_Goal%,Field_Goal_home%,Field_Goal_away%
0,ARI,0.363636,0.636364,0.400000,0.333333
1,ATL,0.700000,0.300000,0.800000,0.600000
2,BAL,0.090909,0.909091,0.000000,0.200000
3,BUF,0.545455,0.454545,0.428571,0.750000
4,CAR,0.300000,0.700000,0.400000,0.200000


In [81]:
first_score_df.to_excel('first_score_fg.xlsx', index=True)

In [82]:
# add schedule 
# create DF for upcoming week
# create stats to show upcpoming games
grouped = nfl_data.groupby('week')

# Select the group corresponding to week 6
week_6_data = grouped.get_group(12)
get_teams =['home_team','away_team']
schedule_week_df = week_6_data[get_teams]
schedule_week_df.reset_index(drop=True, inplace=True)
schedule_week_df.head()

,home_team,away_team
0,DET,GB
1,DAL,WAS
2,SEA,SF
3,NYJ,MIA
4,ATL,NO


In [83]:
# add FG % to schedule
# merge home team with home stats
home_team = schedule_week_df.merge(first_score_df, left_on='home_team', right_on='team')
home_team.rename(columns={'Field_Goal%': 'home_team_total_fg%'}, inplace=True)
home_remove = ['Field_Goal_away%', 'team']
home_team = home_team.drop(columns=home_remove)


home_team.head()

,home_team,away_team,home_team_total_fg%,Touchdown_Goal%,Field_Goal_home%
0,DET,GB,0.5,0.5,0.40
1,DAL,WAS,0.2,0.8,0.25
2,SEA,SF,0.2,0.8,0.40
3,NYJ,MIA,0.6,0.4,0.60
4,ATL,NO,0.7,0.3,0.80


In [84]:
# merge home team with home stats
away_team = schedule_week_df.merge(first_score_df, left_on='away_team', right_on='team')
away_team.rename(columns={'Field_Goal%': 'away_team_total_fg%','Touchdown_Goal%': 'away_team_total_td%'}, inplace=True)
away_remove = ['Field_Goal_home%', 'team']
away_team = away_team.drop(columns=away_remove)
away_team.head()

,home_team,away_team,away_team_total_fg%,away_team_total_td%,Field_Goal_away%
0,DET,GB,0.500000,0.500000,0.600000
1,DAL,WAS,0.272727,0.727273,0.333333
2,SEA,SF,0.100000,0.900000,0.000000
3,NYJ,MIA,0.300000,0.700000,0.400000
4,ATL,NO,0.300000,0.700000,0.333333


In [85]:
# merge home and away on each other
first_fg_total = pd.merge(home_team, away_team, on=['home_team', 'away_team'])

first_fg_total.head()

,home_team,away_team,home_team_total_fg%,Touchdown_Goal%,Field_Goal_home%,away_team_total_fg%,away_team_total_td%,Field_Goal_away%
0,DET,GB,0.5,0.5,0.40,0.500000,0.500000,0.600000
1,DAL,WAS,0.2,0.8,0.25,0.272727,0.727273,0.333333
2,SEA,SF,0.2,0.8,0.40,0.100000,0.900000,0.000000
3,NYJ,MIA,0.6,0.4,0.60,0.300000,0.700000,0.400000
4,ATL,NO,0.7,0.3,0.80,0.300000,0.700000,0.333333


In [86]:
first_fg_total.to_excel('fg_week_ten_two.xlsx', index=True)